## Introduction

In this notebook, we will try to obtain the unique headers amongst all the csv files that we are going to merge. Then we will finally make a master comma seperated value file with list of headers that we obtain from this notebook

## Data Cleaning

### Import the necessary libaries 

In [1]:
import pandas as pd
import numpy as np
import glob
import csv
import re
import os

### Setting up parent directory and sub directory

In [2]:
parent_dir = "../Data/"
filelist = []
dirs = []

def makefilelist(parent_dir):
    headers = []
    csv_headers = []
    subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
    filelist = []
    for dir in subject_dirs:
        csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
        for file in csv_files:
            filelist.append(file)
    
    return filelist, subject_dirs

### Read headers from CSV files

In [21]:
def readCSV(fileList, subject_dirs):
    master_csv_headers = []
    for filename in fileList:
        slash = filename.split("/")
        parts = slash[2]
        subparts = parts.split("_")
        CIK = subparts[0]
        report_type = subparts[1]
        subsubpart = subparts[2].split('-')
        report_year = subsubpart[1]    
        df = pd.read_csv(filename,engine='python')
        df['CIK'] = CIK
        df['Reporting Type'] = report_type
        df['Report Year'] = report_year
        df.to_csv(filename, encoding='utf-8', index=False)
        
        with open(filename, 'r') as f:
            d_reader = csv.DictReader(f)
            headers = d_reader.fieldnames
            for header in headers:
                master_csv_headers.append(header)
            
    return master_csv_headers

In [22]:
filelist, dirs = makefilelist(parent_dir)
csv_headers = readCSV(filelist, dirs)

Now we will only use the unqiue headers that we might require for our master CSV

In [23]:
def uniqueHeaders(csv_headers):
    return set(csv_headers)

def chomp(list1):
    list1 = [x.replace('\n', '') for x in list1]
    return list1

Checking our unique header values and taking a look on the kind of values we have

In [33]:
print(csv_headers)
print(len(csv_headers))
lol = chomp(final_headers)
print(len(lol))

['Counterparty\n', '\nNotional amount', '\nTermination date', '\nUnrealized appreciation(depreciation)', 'CIK', 'Reporting Type', 'Report Year', 'Counterparty', 'Notional amount', 'dates', 'Fixed rate', '(depreciation)', 'CIK', 'Reporting Type', 'Report Year', 'Counterparty', 'Reference Entity/Obligation', 'Buy/Sell Protection', '(Pay)/ Receive Fixed Rate (%)', 'Termination Date', 'Notional Amount(2)', 'Fair Value(3)', 'Upfront Payments Paid/ (Received)', 'Unrealized Appreciation/ (Depreciation)', 'CIK', 'Reporting Type', 'Report Year', 'Reference Entity/Obligation', 'Buy/Sell Protection', '(Pay)/ Receive Fixed Rate (%)', 'Clearinghouse', 'Termination Date', 'Notional Amount(2)', 'Fair Value(3)', 'Unrealized Appreciation/ (Depreciation)', 'CIK', 'Reporting Type', 'Report Year', 'Underlying Instrument', 'Counterparty', 'Amount($)', 'Rate(%)', 'Date', 'CIK', 'Reporting Type', 'Report Year', 'Underlying Instrument', 'Counterparty', 'Amount($)', 'Fixed Rate (%) ', 'Date', 'Value($)', 'CIK'

### Checking for different instances of similar headers

Using this to test out the best conditions required to extract all the possible variants of each header in the unified CSV

In [34]:
counter = 0
for header in lol:
    if 'notional' in header.lower() or 'amount' in header.lower():
        print(header)
        counter+=1
print ("Counter", counter)

NotionalValue
AMOUNT (000 S)
Amount  (4)
 Notional Amount      ($)(c)
Amount(000 s)
AMOUNT
Notional Amount(000)
Notional  Amount  (000 s  omitted)
Amount  (USD)
NotionalAmount(a)
Amount (4)
Notional Amount
Notional Amount ($) (f)
NotionalAmount
Amount(000s)
NotionalAmount(U.S. Dollars)
Notional Value3
Notional   Amount   ($)(c)
Notional Amount2
 Notional Amount
NotionalAmount
Notional Amount ($) (g)
Notional Amount
Notionalamount(000)
NOTIONAL AMOUNT  
NotionalValue
Notional Amount($000)
Notional Amount (1)
Notional
NotionalAmount ($)
Notional  Amount  (000s  omitted)
Notional   Amount(2)
NotionalAmount(000)
Notional   Amount(c) 
NotionalAmount(000)2
Amount (2)
  Notional Amount
Amount 
AMOUNT(000 S)
NotionalAmount($)(c)
Notional Amount(000s)
Notional Amount (000s)
Notional Amount (000)
Notional Amount ($) (k)
NotionalPrincipalAmount2
Notional amount
Notional Amount
NOTIONAL  AMOUNT  
Notional   Amount (3)
AMOUNT(000 s)
Notional  Amount (4)
Notional   Amount 
NOTIONAL AMOUNT2
Notional 

We will usse the above function as a way to guage how many similar ways are there to report a specific header and then merge all the occurances together to the master csv when we are conducting the joining

### Normalizing the values present in the column

In [ ]:
def normalizeValues(filelist):
    for filename in filelist:
        with open(filename, 'r') as f:
            df = pd.read_csv(filename, error_bad_lines=False)
            headers = list(df)
            for header in headers:
                if '000' in header:
                    df[header] = df[header].astype(str) + '000'
            print (df)
            print("Filename", filename)        
            df.to_csv(filename, index=False, header=True)

In [ ]:
normalizeValues(filelist)

### Generate conditions for each type of Header 

In this section, we will decide the final size of the unified CSV that we are going to generate for each time user searches for something. 

In [ ]:
unified_csv = pd.DataFrame()
